In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import itertools
import random
import math

#### Erdös-Rényi (ER) networks

In [2]:
def erdos_renyi_gen(n,p):
    net = nx.Graph()
    for node in range(0,n):
        net.add_node(node)
    
    for ini in range(0,n):
        for end in range(ini+1, n):
            if (p > 0 and random.random() < p):
                net.add_edge(ini, end)
    return net

#### Watts-Strogatz model (WS)

In [3]:
def watts_strogatz_gen(n, k, p):
    if (k >= n): 
        raise Exception('Invalid value for k. k must be less than n')

    net = nx.Graph()
    k_2 = math.floor(k / 2)
    edge_list = []

    for node in range(0,n):
        for node_pos_shift in range(1, k_2 + 1):
            edge_list.append([node, (node + node_pos_shift) % n])

    net.add_edges_from(edge_list)

    for edge in edge_list:
        if (p > 0 and random.random() < p):
            new_to = random.randint(0, n-1)

            while new_to == edge[1] or net.has_edge(edge[0], new_to):
                new_to = random.randint(0, n-1)

            if net.degree(edge[0]) >= n - 1:
                break
            else:
                net.remove_edge(edge[0], edge[1])
                net.add_edge(edge[0], new_to)
    
    return net

##### Barabási-Albert model (BA)

In [4]:
def barabasi_albert_gen(n, m0, m):

    if (m0 > n):
        raise Exception('m0 must be equal or less than n.')

    if (m > m0):
        raise Exception('m must be equal or less than m0.')

    net = nx.Graph()

    for node in range(0,m0):
        #print(f'Adding node {node}')
        net.add_node(node)

    #Randomly connect them. At least one connection per node.
    for node in range(0,m0):
        if (net.degree(node) < m0 - 1):
            targets = list(range(0, m0))
            random.shuffle(targets)
            for node_to in targets:
                if (node != node_to and not net.has_edge(node, node_to) and net.degree(node_to) < m0):
                    #print (f'Adding edge {node} to {node_to}')
                    net.add_edge(node, node_to)
                    break
    while m0 < n:    
        distr_degree = []
        for degree in net.degree():
            distr_degree += [degree[0]] * degree[1]

        for _ in range(0, m):
            node = random.choice(distr_degree)
            distr_degree = [item for item in distr_degree if item != node]

            net.add_node(m0)
            net.add_edge(m0, node)     
        m0 += 1

    return net

In [7]:
k = nx.average_node_connectivity(G)

KeyboardInterrupt: 

In [5]:
N = 10
p = 0.2
G = erdos_renyi_gen(N, p)